In [ ]:
%matplotlib inline

In [ ]:
import itertools
import os
import time

import kwant
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg as la
import scipy.signal as signal
import scipy.sparse.linalg as sla
from qm_tools import *
from tqdm.notebook import tqdm

In [ ]:
plt.style.use("sans_style.mplstyle")
plt.close("all")

In [ ]:
from fhj import (
    fhj_hamiltonian_sym,
    generate_pf_domains,
    generate_pf_tripartite,
    get_default_params,
    get_default_pf_params_domains,
    get_default_pf_params_tripartite,
    make_2D_system,
)
from physics_utils import constants, pauli

constants.m_star = 0.026
constants.c_k = (
    constants.hbar ** 2
    / (2 * constants.m_star * constants.m_e)
    / (constants.eV * 1e-3 * 1e-18)
)

In [ ]:
params = {
    # Material params
    "k_c": 38.0998212 / 0.026,
    # Spin orbit
    "alpha_x": 0,
    "alpha_y": 0,
    "alpha_z": 0,
    # Electrostatics
    "V": lambda *args: 0,
    # Induced terms params
    "Delta": lambda *args: 0,
    "theta": lambda *args: 0,
    # Zeeman field
    "h_x": lambda *args: 0,
    "h_y": lambda *args: 0,
    "h_z": lambda *args: 0,
    # Other
    "cos": np.cos,
    "sin": np.sin,
}

Delta_0 = 0.250

pf_params = dict(
    L_junction=180,
    mu_C=3.0,
    mu_L=1.0,
    mu_R=1.0,
    L_barr_L=0,
    L_barr_R=0,
    V_barr_L=0.5,
    V_barr_R=0.5,
    Delta_0=Delta_0,
    theta_LR=np.pi,
    h_x_L=0.8 * Delta_0,
    h_x_C=0.8 * Delta_0,
    h_x_R=0.8 * Delta_0,
    h_y_L=0.0,
    h_y_C=0.0,
    h_y_R=0.0,
)

V, Delta, theta, h_x, h_y, h_z = generate_pf_tripartite(**pf_params)
params.update(V=V, Delta=Delta, theta=theta, h_x=h_x, h_y=h_y, h_z=h_z)

In [ ]:
L_x = 6000  # Length in nanometers
a_x = 4  # Discretization step

L_y = 20  # Width in nanometers
a_y = 1000  # Discretization step. Setting a_y to be much bigger to L_y creates a 1D system

In [ ]:
x = np.linspace(-400, 400, 100)

fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, figsize=(4, 8), dpi=100)

ax1.plot(x, V(x, 0), "-C0", label=r"$V$")
ax1b = ax1.twinx()
ax1b.plot(x, Delta(x, 0), "-C1", label=r"$\Delta$")
ax1b.plot(x, h_x(x, 0), "-C2", label=r"$h_x$")
ax1b.plot(x, h_y(x, 0), "-C3", label=r"$h_y$")
ax1b.legend()

# ax2.twinx().plot(x, theta(x, 0), "-C2")

# ax3.plot(x, h_x(x, 0))

In [ ]:
theta_N = 60
theta_ax = np.linspace(0, 2 * np.pi, theta_N, endpoint=False)  # + 0.001
dtheta = theta_ax[1]

k = 100

syst, lat, hop = make_2D_system(L_x=L_x, L_y=L_y, a_x=a_x, a_y=a_y, with_leads=False)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 2))
kwant.plot(syst, ax=ax)

ax.set_xlim(-200, 200)

In [ ]:
def diag_theta(params, theta_ax, k):

    theta_N = len(theta_ax)

    ws = []
    vs = []

    for n in tqdm(range(theta_N)):

        theta_v = theta_ax[n]
        params["theta"] = lambda x, y: theta_v * np.heaviside(x, 0)

        ham = syst.hamiltonian_submatrix(params=params, sparse=True)
        w, v = sla.eigsh(ham, sigma=0, which="LM", k=k)

        idxs = np.argsort(w)
        ws.append(w[idxs])
        vs.append(v[:, idxs])

    ws = np.array(ws)
    vs = np.array(vs)

    return ws, vs

In [ ]:
ws, vs = diag_theta(params, theta_ax, k=k)
ws, vs = sort_eigensystem(ws, vs)

In [ ]:
ie = np.diff(ws, axis=0)

In [ ]:
fig, ax = plt.subplots(figsize=(3.375, 2))
for n in range(40, 50):
    ax.plot(theta_ax[:-1], ie[:, n] / Delta_0, '-')
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(figsize=(3.375, 2))
for n in range(40, 50):
    ax.plot(theta_ax, ws[:, n] / Delta_0, '-')
ax.set_ylim(-3, 3)
fig.tight_layout()

In [ ]:
W = kwant.operator.Density(syst, np.kron(pauli.s0, pauli.s0)/4)
W_bound = W.bind(params=params)

S_x = kwant.operator.Density(syst, np.kron(pauli.sz, pauli.sx))
S_x_bound = S_x.bind(params=params)

In [ ]:
wf = []
for i in range(theta_N):
    wfi = []
    for n in range(k):
        wfi.append(W_bound(vs[i, :, n]))
    wf.append(np.array(wfi))
wf = np.array(wf)

In [ ]:
x = np.linspace(-L_x/2, L_x/2, wf.shape[-1])

n_idx = np.argmin(np.abs(ws[0]))

fig, ax = plt.subplots()
ax.plot(x, wf[0, n_idx], '-')
ax.set_xlim(-300, 300)
    
ax2 = ax.twinx()
ax2.plot(x, V(x, 0), 'C1-')

In [ ]:
spins = np.zeros_like(ws)
for i in range(theta_N):
    for n in range(k):
        spins[i, n] = np.sum(S_x(vs[i, :, n]))

In [ ]:
fig, ax = plt.subplots(figsize=(3.375, 2))
for n in range(k):
    ax.scatter(theta_ax, ws[:, n] / Delta_0, s=0.1, c=spins[:, n], cmap="RdBu", vmax=1, vmin=-1)
ax.set_ylim(-3, 3)
fig.tight_layout()